In [9]:
%matplotlib inline  
from fastai.vision import *
from fastai.data_block import _maybe_squeeze
from fastai.callbacks import *
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from efficientnet_pytorch import EfficientNet
from ranger import *
from mxresnet import *

In [10]:
def strt_split(x, y, n_folds=5, random_seed = 42, path=Path('')):  
    try: 
        val_name = load('val_idx.joblib')
    except:
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
        val_name = [(val_idx, trn_idx) for trn_idx, val_idx in skf.split(x, y)]
        dump(val_name,'val_idx.joblib')
    return val_name

def modified_label_from_df(self, cols:IntsOrStrs=1, label_cls:Callable=None, **kwargs):
    "Label `self.items` from the values in `cols` in `self.inner_df`."
    self.inner_df.labels.fillna('', inplace=True)
    labels = self.inner_df.iloc[:,df_names_to_idx(cols, self.inner_df)]
    assert labels.isna().sum().sum() == 0, f"You have NaN values in column(s) {cols} of your dataframe, please fix it."
    if is_listy(cols) and len(cols) > 1 and (label_cls is None or label_cls == MultiCategoryList):
        new_kwargs,label_cls = dict(one_hot=True, classes= cols),MultiCategoryList
        kwargs = {**new_kwargs, **kwargs}
    return self._label_from_list(_maybe_squeeze(labels), label_cls=label_cls, **kwargs)


def flattenAnneal(learn:Learner, lr:float, n_epochs:int, start_pct:float, SUFFIX = 'PHASE_1_COS'):
    n = len(learn.data.train_dl)
    anneal_start = int(n*n_epochs*start_pct)
    anneal_end = int(n*n_epochs) - anneal_start
    phases = [TrainingPhase(anneal_start).schedule_hp('lr', lr),
             TrainingPhase(anneal_end).schedule_hp('lr', lr, anneal=annealing_cos)]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    learn.callbacks.append(SaveModelCallback(learn, every='improvement', monitor='valid_loss', name = f'{EXP_NAME}_{SUFFIX}'))
    learn.fit(n_epochs)

In [11]:
PATH = Path('..')
FOLD =0
EXP_NAME =      f'NB_EXP_10_CV_{FOLD}'
IMG_TRAIN_224 = PATH/'train_images_224'
IMG_TEST_224  = PATH/'test_images_224'
DF_TRAIN =      pd.read_csv(PATH/'train_labels_as_strings.csv')
DF_SUBMI =      pd.read_csv(PATH/'stage_1_sample_submission.csv')
BS =            384
SZ =            224



DF_SUBMI['fn'] = DF_SUBMI.ID.apply(lambda x: '_'.join(x.split('_')[:2]) + '.png')
DF_TRAIN['labels'].fillna('', inplace=True)
VAL_IDX = strt_split(DF_TRAIN['fn'], DF_TRAIN['labels'])[FOLD]

In [12]:
ItemList.label_from_df = modified_label_from_df
test_fns = DF_SUBMI.fn.unique()

data = (ImageList.from_csv('..', 'train_labels_as_strings.csv', folder=IMG_TRAIN_224.name)
        .split_by_idxs(valid_idx=VAL_IDX[0], train_idx=VAL_IDX[1])
        .label_from_df(label_delim=' ')
        .transform(tfms = get_transforms(),size=SZ)
        .add_test('../' +IMG_TEST_224.name + '/' + test_fns)
        .databunch(bs=BS))

        

In [13]:
md_xrsa =  mxresnet50(c_out=data.c, sa=True)
opt_func = partial(Ranger, betas=(0.95,0.99), eps=1e-6)

In [6]:
learn = Learner(data,
                md_xrsa,
                wd=1e-2,
                bn_wd=False, 
                true_wd=True, 
                opt_func=opt_func,
                metrics=[accuracy_thresh])

learn.model = nn.DataParallel(learn.model)
learn.to_fp16()
learn.unfreeze()

In [ ]:
learn.mixup(stack_y=False)

In [ ]:
lr = 1e-2/3
learn.recorder.plot(skip_end=1)
plt.axvline(lr)

In [7]:
lr = 1e-2/3
flattenAnneal(learn, lr, 20, 0.7)

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.096379,0.094411,0.967433,20:37
1,0.084853,0.080725,0.971553,20:40
2,0.079183,0.079181,0.972300,20:42
3,0.073785,0.074474,0.974261,20:42
4,0.071993,0.070438,0.975138,20:46
5,0.070687,0.069346,0.975460,20:24
6,0.067750,0.067530,0.976002,20:43
7,0.066519,0.066805,0.976321,20:44
8,0.067872,0.064937,0.976587,20:44
9,0.065047,0.065223,0.976740,20:44


Better model found at epoch 0 with valid_loss value: 0.09441084414720535.
Better model found at epoch 1 with valid_loss value: 0.08072510361671448.
Better model found at epoch 2 with valid_loss value: 0.07918119430541992.
Better model found at epoch 3 with valid_loss value: 0.07447420060634613.
Better model found at epoch 4 with valid_loss value: 0.07043765485286713.
Better model found at epoch 5 with valid_loss value: 0.06934646517038345.
Better model found at epoch 6 with valid_loss value: 0.06752996891736984.
Better model found at epoch 7 with valid_loss value: 0.06680489331483841.
Better model found at epoch 8 with valid_loss value: 0.06493663042783737.
Better model found at epoch 10 with valid_loss value: 0.06347418576478958.
Better model found at epoch 11 with valid_loss value: 0.06286603212356567.
Better model found at epoch 13 with valid_loss value: 0.06266184896230698.
Better model found at epoch 14 with valid_loss value: 0.06120621785521507.
Better model found at epoch 15 wit

In [ ]:
def get_preds(learn:Learner, sub_fn: str=f'{EXP_NAME}_COS', TTA: bool = False, dt_type = DatasetType.Test):
    if TTA:
        learn.to_fp32()
        preds, targs = learn.TTA(ds_type=dt_type)
        sub_fn = f'{sub_fn}_TTA'
    else:
        preds, targs = learn.get_preds(dt_type)
    ids = []
    labels = []

    for fn, pred in zip(test_fns, preds):
        for i, label in enumerate(data.train_ds.classes):
            ids.append(f"{fn.split('.')[0]}_{label}")
            predicted_probability = '{0:1.10f}'.format(pred[i].item())
            labels.append(predicted_probability)
    pd.DataFrame({'ID': ids, 'Label': labels}).to_csv(f'{sub_fn}.csv', index=False)

In [ ]:
get_preds(learn)

In [ ]:
get_preds(learn, TTA=True)

In [ ]:
!sudo shutdown